<a href="https://colab.research.google.com/github/ileeee-sh/2025-HUFS-College-of-English-_-HUFSTAG/blob/main/%ED%95%99%EC%88%A0%EC%A0%9C_%EA%B0%90%EC%84%B1%EB%B6%84%EC%84%9D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import re
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from io import StringIO
import os
!pip install langdetect
from langdetect import detect, DetectorFactory
import nltk



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 15.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993223 sha256=18b301782934a76030d4ab54fa6e0c6ef5e794e2b561d12f8e31a826f346d23d
  Stored in directory: /root/.cache/pip/wheels/c1/67/88/e844b5b022812e15a52e4eaa38a1e709e99f06f6639d7e3ba7
Successfully built langdetect


In [ ]:
# 라이브러리 설치 및 NLTK 데이터 다운로드
try:
    print("필요 라이브러리 설치 중...")
    # pandarallel 설치
    get_ipython().system('pip install pandas openpyxl nltk langdetect pandarallel')
except NameError:
    print("Colab 환경이 아닌 경우, 'pip install ...' 명령어를 실행해야 합니다.")

# langdetect의 비결정적 동작을 방지하기 위해 시드를 설정
DetectorFactory.seed = 42

# NLTK 리소스 다운로드
print("NLTK 데이터 다운로드 중...")
try:
    nltk.download('punkt', quiet=True)
    nltk.download('vader_lexicon', quiet=True)
    print("NLTK 데이터 다운로드 완료.")
except Exception as e:
    print(f"NLTK 다운로드 오류: {e}")



필요 라이브러리 설치 중...
NLTK 데이터 다운로드 중...
NLTK 데이터 다운로드 완료.


In [ ]:
try:
    from pandarallel import pandarallel
    # pandarallel 초기화
    pandarallel.initialize(progress_bar=True)
    PANDARALLEL_READY = True
    print("\n[성능 최적화] pandarallel 초기화 성공. 언어 감지에 병렬 처리를 사용합니다.")
except Exception as e:
    # 초기화에 실패할 경우 순차 처리 사용
    print(f"\n[성능 최적화] pandarallel 초기화 실패: {e}. 순차 처리를 사용합니다.")
    PANDARALLEL_READY = False

# 설정 변수
COMMENT_COLUMN = 'Comment_Content'
KEYWORD_COLUMN_NAME = 'Keyword'




INFO: Pandarallel will run on 1 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.

[성능 최적화] pandarallel 초기화 성공. 언어 감지에 병렬 처리를 사용합니다.


In [ ]:
def load_custom_stopwords(filepath):
    """지정된 파일에서 사용자 정의 불용어 목록을 로드하거나 Colab 업로드 기능을 사용합니다."""
    # 기본 불용어는 NLTK 영어 불용어로 시작
    custom_stopwords = set(stopwords.words('english'))

    # Colab 환경 체크 및 파일 업로드 요청
    try:
        from google.colab import files
        print("\n[불용어 파일 업로드] 'custom_stopwords.txt' 파일을 업로드해 주세요.")
        uploaded = files.upload()

        if not uploaded:
            print(f"경고: 업로드된 파일이 없습니다. 기본 NLTK 불용어 ({len(custom_stopwords)}개)를 사용합니다.")
            return custom_stopwords

        # 첫 번째 업로드된 파일을 사용
        uploaded_filename = list(uploaded.keys())[0]
        print(f"업로드된 파일 '{uploaded_filename}'에서 불용어를 로드합니다.")

        # 업로드된 파일의 내용을 문자열로 읽어와 불용어 세트를 만듭니다.
        file_content = uploaded[uploaded_filename].decode('utf-8')
        custom_stopwords = set(line.strip().lower() for line in file_content.splitlines() if line.strip())

    except Exception as e:
        print(f"불용어 로드 중 오류 발생: {e}. 기본 NLTK 불용어를 사용합니다.")

    print(f"총 {len(custom_stopwords)}개의 사용자 정의 불용어 로드 완료.")
    return custom_stopwords



In [ ]:
def detect_language(text):
    if not isinstance(text, str):
        return 'too_short'

    clean_text = str(text).strip()

    # 짧은 텍스트(3단어 미만 또는 10자 미만)는 오류 방지 및 분석 포함을 위해 'en'으로 강제 처리
    if len(clean_text.split()) < 3 or len(clean_text) < 10:
        return 'en'

    try:
        return detect(clean_text)
    except:
        return 'error' # 감지 오류 발생 시



In [ ]:
def preprocess_text(text, custom_stopwords):
    """텍스트 전처리: 소문자화, @mention/URL/해시태그/이모지 제거, 구두점 제거, 불용어 제거, 토큰화."""
    if not isinstance(text, str):
        return ""

    # 1. 소문자화
    text = text.lower()

    # 2. @mention 제거
    text = re.sub(r'@\w+', '', text)

    # 3. URL 제거
    text = re.sub(r'https?://\S+', '', text, flags=re.MULTILINE)

    # 4. 해시태그 제거
    text = re.sub(r'#\w+', '', text)

    # 5. 이모지 제거 (유니코드 범위 사용)
    emoji_pattern = re.compile(
        "["
        "\U0001F600-\U0001F64F"  # Emoticons
        "\U0001F300-\U0001F5FF"  # Symbols & Pictographs
        "\U0001F680-\U0001F6FF"  # Transport & Map Symbols
        "\U0001F1E0-\U0001F1FF"  # Flags (iOS)
        "\U00002702-\U000027B0"
        "\U000024C2-\U0001F251"
        "]+", flags=re.UNICODE
    )
    text = emoji_pattern.sub(r'', text)

    # 6. 특수 문자 및 구두점 제거 (영어 알파벳과 공백만 남김)
    text = re.sub(r'[^\w\s]', '', text)

    # 7. 토큰화 (단어로 분리)
    tokens = word_tokenize(text)

    # 8. 불용어 및 짧은 단어(길이 1) 제거
    processed_tokens = [
        word for word in tokens
        if word not in custom_stopwords and len(word) > 1
    ]

    return " ".join(processed_tokens)



In [ ]:
# NLTK 리소스 다운로드
import nltk
nltk.download('punkt')
nltk.download('vader_lexicon')
nltk.download('stopwords')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
import nltk
nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('stopwords')
nltk.download('vader_lexicon')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [ ]:
def perform_sentiment_analysis(input_excel_path, stopwords_file_path):
    """
    엑셀 파일을 로드하고, 언어 감지 및 전처리 후 VADER 감정 분석을 수행합니다.
    """
    try:
        from google.colab import files

        # 1. 데이터 파일 업로드 요청
        if not os.path.exists(input_excel_path):
            print(f"\n[데이터 파일 업로드] 데이터 파일 '{input_excel_path}'을(를) 찾을 수 없습니다. 파일을 업로드해 주세요.")
            data_uploaded = files.upload()

            # 업로드  확인
            if input_excel_path not in data_uploaded:
                # 파일명이 다를 경우 첫 번째 업로드된 파일을 사용하도록 로직 변경
                if data_uploaded:
                    uploaded_key = list(data_uploaded.keys())[0]
                    if uploaded_key.endswith('.xlsx') or uploaded_key.endswith('.xlsm'):
                        input_excel_path = uploaded_key
                        print(f"업로드된 파일 '{uploaded_key}'를 데이터 파일로 사용합니다.")
                    else:
                        print("오류: 업로드된 파일이 Excel 파일 형식이 아닙니다.")
                        return
                else:
                    print("오류: 데이터 파일 업로드가 취소되었습니다.")
                    return

        # 2. 데이터 로드
        print(f"\n[데이터 로드] 입력 파일 '{input_excel_path}'의 모든 시트를 로드 중...")
        df_dict = pd.read_excel(input_excel_path, engine='openpyxl', sheet_name=None)

        if not df_dict:
            print(f"오류: 입력 파일 '{input_excel_path}'에서 시트를 찾을 수 없습니다.")
            return

        # 모든 시트의 DataFrame을 하나의 DataFrame으로 병합
        df = pd.concat(df_dict.values(), ignore_index=True)

        # 병합된 셀 처리: NaN 값을 앞의 유효한 값으로 채웁니다.
        df[KEYWORD_COLUMN_NAME] = df[KEYWORD_COLUMN_NAME].fillna(method='ffill')

    except FileNotFoundError:
        print(f"오류: 입력 파일 '{input_excel_path}'을(를) 찾을 수 없습니다.")
        return
    except Exception as e:
        print(f"데이터 로드 중 오류 발생: {e}")
        return

    # 필요한 열 존재 확인
    required_cols = [KEYWORD_COLUMN_NAME, COMMENT_COLUMN]
    for col in required_cols:
        if col not in df.columns:
            print(f"오류: 엑셀 파일에 필요한 열 ('{col}')이 없습니다. 감지된 열: {df.columns.tolist()}")
            return

    print(f"총 {len(df)}개 데이터 로드 완료 (모든 시트 병합).")

    # 3. 언어 감지
    print("언어 감지 수행 중...")

    # pandarallel 사용 여부에 따라 처리 방식 분기
    global PANDARALLEL_READY
    if PANDARALLEL_READY:
         # 병렬 처리 적용
         df['detected_language'] = df[COMMENT_COLUMN].parallel_apply(detect_language)
    else:
         # 순차 처리 적용
         df['detected_language'] = df[COMMENT_COLUMN].apply(detect_language)

    # 영어 코멘트만 필터링
    df_english = df[df['detected_language'] == 'en'].copy()
    print(f"영어(en)로 감지된 코멘트 {len(df_english)}개 필터링 완료.")

    if df_english.empty:
        print("분석할 유효한 영어 코멘트가 없습니다.")
        return

    # 4. 사용자 정의 불용어 로드
    custom_stopwords = load_custom_stopwords(stopwords_file_path)

    # 5. 텍스트 전처리 (필터링된 영어 코멘트에만 적용)
    print("텍스트 전처리 중...")
    df_english['processed_text'] = df_english[COMMENT_COLUMN].apply(lambda x: preprocess_text(x, custom_stopwords))

    # 6. 최종 유효성 검사 (전처리 후 텍스트가 남았는지 확인)
    df_final = df_english[df_english['processed_text'].str.len() > 0].copy()
    print(f"전처리 후 최종 분석할 코멘트 {len(df_final)}개 준비 완료.")

    if df_final.empty:
        print("전처리 후 분석할 유효한 텍스트가 남아있지 않습니다.")
        return

    # 7. VADER 감정 분석기 초기화 및 점수 계산
    sid = SentimentIntensityAnalyzer()
    print("VADER 감정 분석 수행 중...")
    df_final['scores'] = df_final['processed_text'].apply(sid.polarity_scores)

    df_final['neg'] = df_final['scores'].apply(lambda x: x['neg'])
    df_final['neu'] = df_final['scores'].apply(lambda x: x['neu'])
    df_final['pos'] = df_final['scores'].apply(lambda x: x['pos'])
    df_final['compound'] = df_final['scores'].apply(lambda x: x['compound'])

    # 감정 레이블 분류
    def get_sentiment(score):
        if score >= 0.05: return 'Positive'
        elif score <= -0.05: return 'Negative'
        else: return 'Neutral'

    df_final['sentiment'] = df_final['compound'].apply(get_sentiment)

    # 8. 결과 출력 (VADER 4가지 점수의 평균 출력)
    print("\n" + "="*50)
    print("최종 분석 결과")
    print("="*50)

    print("\n--- 전체 분석 결과 요약 (복합 점수 기준) ---")
    print(df_final['sentiment'].value_counts())

    print("\n--- 해시태그(Keyword)별 VADER 점수 평균 ---")

    # Keyword 별로 4가지 VADER 점수 평균을 계산
    score_cols = ['neg', 'neu', 'pos', 'compound']
    keyword_summary = df_final.groupby(KEYWORD_COLUMN_NAME)[score_cols].mean()

    # Keyword별 코멘트 수 (Comment_Count) 추가
    keyword_summary['Comment_Count'] = df_final.groupby(KEYWORD_COLUMN_NAME).size()

    # 소수점 셋째 자리까지 반올림
    keyword_summary = keyword_summary.round(3)

    # compound를 기준으로 내림차순 정렬
    keyword_summary = keyword_summary.sort_values(by='compound', ascending=False)

    # 열 이름 변경
    keyword_summary.columns = ['Avg_Negative_Score', 'Avg_Neutral_Score', 'Avg_Positive_Score', 'Avg_Compound_Score', 'Comment_Count']

    print(keyword_summary)


    print("\n--- 상위 5개 긍정 코멘트 (Keyword 포함) ---")
    top_positive = df_final.sort_values(by='compound', ascending=False).head(5)
    print(top_positive[[KEYWORD_COLUMN_NAME, COMMENT_COLUMN, 'compound', 'sentiment']].to_string(index=False))

    print("\n--- 상위 5개 부정 코멘트 (Keyword 포함) ---")
    top_negative = df_final.sort_values(by='compound', ascending=True).head(5)
    print(top_negative[[KEYWORD_COLUMN_NAME, COMMENT_COLUMN, 'compound', 'sentiment']].to_string(index=False))

    # 9. 결과 저장(.xlsx)
    output_path = 'Sentiment_Analysis_Results.xlsx'

    # Keyword 요약 시트를 추가하여 엑셀 파일에 저장
    output_cols = [KEYWORD_COLUMN_NAME, COMMENT_COLUMN, 'detected_language', 'processed_text', 'neg', 'neu', 'pos', 'compound', 'sentiment']

    with pd.ExcelWriter(output_path, engine='openpyxl') as writer:
        # 모든 코멘트 상세 결과 시트
        df_final.to_excel(writer, index=False, sheet_name='All_Comments', columns=output_cols)
        # Keyword별 요약 시트
        keyword_summary.to_excel(writer, sheet_name='Keyword_Summary')

    print(f"\n분석 결과가 '{output_path}'에 2개의 시트(All_Comments, Keyword_Summary)로 저장되었습니다.")

    # 엑셀 다운로드 자동화
    from google.colab import files
    files.download(output_path)

INPUT_FILE = 'Comments.xlsm'
STOPWORDS_FILE = 'custom_stopwords.txt'
perform_sentiment_analysis(INPUT_FILE, STOPWORDS_FILE)


[데이터 로드] 입력 파일 'Comments.xlsm'의 모든 시트를 로드 중...
총 7653개 데이터 로드 완료 (모든 시트 병합).
언어 감지 수행 중...


/tmp/ipython-input-3127328882.py:40: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df[KEYWORD_COLUMN_NAME] = df[KEYWORD_COLUMN_NAME].fillna(method='ffill')


영어(en)로 감지된 코멘트 5581개 필터링 완료.

[불용어 파일 업로드] 'custom_stopwords.txt' 파일을 업로드해 주세요.


Saving English_Stopwords.xlsx to English_Stopwords (1).xlsx
업로드된 파일 'English_Stopwords (1).xlsx'에서 불용어를 로드합니다.
불용어 로드 중 오류 발생: 'utf-8' codec can't decode byte 0x82 in position 16: invalid start byte. 기본 NLTK 불용어를 사용합니다.
총 198개의 사용자 정의 불용어 로드 완료.
텍스트 전처리 중...
전처리 후 최종 분석할 코멘트 4826개 준비 완료.
VADER 감정 분석 수행 중...

최종 분석 결과

--- 전체 분석 결과 요약 (복합 점수 기준) ---
sentiment
Positive    3143
Neutral     1120
Negative     563
Name: count, dtype: int64

--- 해시태그(Keyword)별 VADER 점수 평균 ---
                      Avg_Negative_Score  Avg_Neutral_Score  \
Keyword                                                       
#BirthdayVibes                     0.008              0.381   
#BirthdayBoy                       0.009              0.311   
#instamood                         0.042              0.245   
#hottrend                          0.000              0.506   
#BirthdayCelebration               0.006              0.369   
#HappyBornDay                      0.011              0.356   
#BirthdayGirl         

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import pandas as pd

df = pd.read_excel("Comments - 복사본.xlsm", engine="openpyxl", sheet_name=0)
print(df.columns.tolist())


['Unnamed: 0', 'Beautiful good morning with best wishes ♥🌿']
